# 1. Chapter : 공간데이터 다루기
`geocoding`과 `table joins`을 통해 위치를 찾고 공간조인을 진행

## 1-1. Geocoding
지오코딩은 이름 혹은 주소를 입력받아서 찾고자 하는 지역 혹은 건물을 찾을 수 있다.\
Google Maps, Bing Maps, Baidu Maps 에서 활용이 가능하다.

### 1) 데이터 불러오기

In [27]:
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
from folium import Marker
import warnings
warnings.filterwarnings('ignore')

# geocoding의 소프트웨어를 불러올 수 있는 Nominatim 라이브러리 불러오기
from geopy.geocoders import Nominatim

- `geopy`는 파이썬의 지오코딩 라이브러리
- `Nominatim`은 OSM 기반의 지오코딩 도구

In [58]:
geolocator = Nominatim(user_agent = 'GIS_learn')
# location = geolocator.geocode('Pyramid of Khufu')
# location2 = geolocator.geocode("seoul city hall")

print(location.point)
print(location.address)
print("---------------")
print(location2.point)
print(location2.address)


29 58m 44.9929s N, 31 8m 3.16788s E
هرم خوفو, Cheops-Osirion-Sphinx Tunnel, كوم الأخضر, الجيزة, 12125, مصر
---------------
37 34m 0.44112s N, 126 58m 42.3134s E
서울특별시청, 110, 세종대로, 태평로1가, 명동, 중구, 서울특별시, 04524, 대한민국


- `geolocator`에 `Nominatim`도구를 인스턴스로 할당하여 찾고자 하는 지점의 포인트 좌표와 주소를 얻을 수 있다.

In [9]:
# 지점의 포인트 좌표 확인
point = location.point
print("Latitude : ", point.latitude)
print("Longitude : ", point.longitude)

Latitude :  29.9791647
Longitude :  31.1342133


다양한 주소를 포함해서 geocoding을 진행하는 방법 \
예시 데이터로 유럽의 상위 100개 대학교 데이터를 불러와서 작업 진행

In [13]:
# 유럽의 상위 100개 대학교 정보 불러오기
universities = pd.read_csv("../../01_data/top_universities.csv")
universities.head(3)

,Name
0,University of Oxford
1,University of Cambridge
2,Imperial College London


### 2) 다수의 행 지오코딩 적용하기

이후 람다함수를 이용해서 데이터 프레임의 모든 행을 지오코딩 할 수 있다. \
(지오코딩이 실패할 경우를 대비해서 try/except로 탈출할 수 있도록 설정)

In [25]:
# geocoding 함수 선언
def my_geocoder(row):
    try:
        point = geolocator.geocode(row).point # 모든 행을 돌면서 Name에 있는 값 검색
        # 학교를 찾을 수 있으면 위도, 경도를 포인트로 받아서 시리즈로 변환하고
        return pd.Series({'Latitude' : point.latitude, 'Longitude':point.longitude})
    except:
        # 찾을 수 없다면 None으로 넘기기
        return None

# universities 데이터프레임에 위도 경도 컬럼을 새롭게 생성
# 람다함수로 my_geocoder 함수를 각 행마다 실행
universities[['Latitude', 'Longitude']] = universities.apply(lambda x: my_geocoder(x['Name']), axis=1)

# 얼만큼 변환이 완료되었는지 비율로 확인
print(f"{(1 - (sum(universities['Latitude'].isna()) / len(universities)))*100}% of address were geoencoded")

# 지오코딩이 실패한 데이터는 삭제
universities = universities.loc[~np.isnan(universities["Latitude"])]

# 지오데이터프레임 형태로 변환
universities = gpd.GeoDataFrame(universities, geometry=gpd.points_from_xy(universities.Longitude, universities.Latitude))

# 좌표계 정의
universities.set_crs(epsg=4326)

universities.head()

,Name,Latitude,Longitude,geometry
0,University of Oxford,51.758707,-1.255669,POINT (-1.25567 51.75871)
1,University of Cambridge,52.189092,0.121298,POINT (0.1213 52.18909)
2,Imperial College London,51.498959,-0.175641,POINT (-0.17564 51.49896)
3,ETH Zurich,47.413218,8.537491,POINT (8.53749 47.41322)
4,UCL,50.851809,4.453697,POINT (4.4537 50.85181)


### 3) 지도에 표현하기
지오코딩 완료 후, 지오데이터프레임 형태로 변환된 데이터를 지도에 시각화

In [28]:
# 배경맵 설정
m_1 = folium.Map(location=[54, 15], tiles='openstreetmap', zoom_start=2)

# 배경맵에 학교 위치를 포인트로 표현하기
for idx, row in universities.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=row['Name']).add_to(m_1)
    
m_1

## 1-2. Table Joins (테이블 조인)

### 1) Attribute Join
여러 DataFrame에서 공통된 인덱스를 기준으로 결합하기 위해서는 `pd.DAtaFrame.join()`을 사용하고 있으며, 이와 같이 인덱스 값을 단순히 일치시켜 데이터를 결합하는 방식을 `Attribute join(속성조인)`으로 호칭함

In [48]:
# 데이터 불러오기
world = gpd.read_file("../../01_data/110m_cultural/ne_110m_admin_0_countries.shp")

# 유럽 데이터만 불러오기
europe = world.loc[world.CONTINENT == 'Europe'].reset_index(drop=True)

# 국가명, 인구수, gdp 데이터는 stasts 데이터프레임으로 저장
europe_stats = europe[['NAME', 'POP_EST', 'GDP_MD']]

# 유럽 경계 데이터는 boundaries 데이터프레임으로 저장
europe_boundaries = europe[['NAME', 'geometry']]

display(europe_stats.head(2))
display(europe_boundaries.head(2))

,NAME,POP_EST,GDP_MD
0,Russia,144373535.0,1699876
1,Norway,5347896.0,403336


,NAME,geometry
0,Russia,"MULTIPOLYGON (((178.7253 71.0988, 180 71.51571..."
1,Norway,"MULTIPOLYGON (((15.14282 79.67431, 15.52255 80..."


각 국가별로 추정된 인구수, GDP(국내총생산) 데이터를 geometry 경계 데이터와 결합을 진행

In [50]:
# 속성 조인을 사용
europe = europe_boundaries.merge(europe_stats, on='NAME')
europe.head()

,NAME,geometry,POP_EST,GDP_MD
0,Russia,"MULTIPOLYGON (((178.7253 71.0988, 180 71.51571...",144373535.0,1699876
1,Norway,"MULTIPOLYGON (((15.14282 79.67431, 15.52255 80...",5347896.0,403336
2,France,"MULTIPOLYGON (((-51.6578 4.15623, -52.24934 3....",67059887.0,2715518
3,Sweden,"POLYGON ((11.02737 58.85615, 11.46827 59.43239...",10285453.0,530883
4,Belarus,"POLYGON ((28.17671 56.16913, 29.22951 55.91834...",9466856.0,63080


### 2) Spatial join (공간 조인)
또 다른 조인 방식은 **공간 조인(spatial join)** 으로, 지오데이터프레임의 공간적 특성을 활용하여 데이터를 결합하는 방법입니다.\
이 방식은 "geometry" 컬럼과 같이 공간 정보를 포함하는 컬럼이 있어야 하며, 이를 통해 두 지오데이터 간의 공간 관계(예: 포함 여부, 교차 여부 등) 를 기준으로 데이터를 연결합니다.\
각 대학이 어느 국가에 속하는지를 알아내기 위해 공간 조인을 사용할 수 있으며, `gpd.sjoin()`함수를 사용하여 공간 조인을 진행

In [54]:
# 유럽에 있는 대학 데이터와 공간조인을 진행
european_universities = gpd.sjoin(universities, europe)

# 결과 확인
print(f"총 {len(universities)}개의 대학이 지오코딩에 성공했으며,")
print(f"그 중에서 {len(european_universities)}개의 대학이 유럽에 존재한다.({len(european_universities.NAME.unique())}개의 서로 다른 국가에 대학들이 존재함.)")

european_universities.head()

총 91개의 대학이 지오코딩에 성공했으며,
그 중에서 87개의 대학이 유럽에 존재한다.(15개의 서로 다른 국가에 대학들이 존재함.)


,Name,Latitude,Longitude,geometry,index_right,NAME,POP_EST,GDP_MD
0,University of Oxford,51.758707,-1.255669,POINT (-1.25567 51.75871),28,United Kingdom,66834405.0,2829108
1,University of Cambridge,52.189092,0.121298,POINT (0.1213 52.18909),28,United Kingdom,66834405.0,2829108
2,Imperial College London,51.498959,-0.175641,POINT (-0.17564 51.49896),28,United Kingdom,66834405.0,2829108
3,ETH Zurich,47.413218,8.537491,POINT (8.53749 47.41322),19,Switzerland,8574832.0,703082
4,UCL,50.851809,4.453697,POINT (4.4537 50.85181),21,Belgium,11484055.0,533097


`gpd.sjoin()` 메서드는 how와 op 인자를 통해 다양한 방식으로 조인 동작을 설정할 수 있습니다.
예를 들어, how='left' 또는 how='right'로 설정하면 SQL의 left join 또는 right join 과 동일한 동작을 수행할 수 있습니다.

# 2. Exercise
스타벅스 데이터 분석가로서 캘리포니아에 새롭게 생길 Starbucks Reserve 매장의 최적에 위치를 탐색

In [59]:
import math
import pandas as pd
import numpy as np
import geopandas as gpd
from geopy.geocoders import Nominatim
import folium
from folium import Marker
from folium.plugins import MarkerCluster

## 2-1. 결측된 지점 geocoding

### 1) 데이터 불러오기

In [62]:
starbucks = pd.read_csv("../../01_data/starbucks_locations.csv")
starbucks.head(2)

,Store Number,Store Name,Address,City,Longitude,Latitude
0,10429-100710,Palmdale & Hwy 395,14136 US Hwy 395 Adelanto CA,Adelanto,-117.40,34.51
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16


In [73]:
# 정보가 결측된 지점 찾기
print(starbucks.isna().sum())

# 결측 데이터가 있는 스타벅스 매정 정보 확인하기
# starbucks[starbucks['Latitude'].isna() | starbucks['Longitude'].isna()]

# 결측된 지점을 별도 데이터프레임으로 저장
rows_with_missing = starbucks[starbucks['City'] == 'Berkeley']

rows_with_missing

Store Number    0
Store Name      0
Address         0
City            0
Longitude       5
Latitude        5
dtype: int64


,Store Number,Store Name,Address,City,Longitude,Latitude
153,5406-945,2224 Shattuck - Berkeley,2224 Shattuck Avenue Berkeley CA,Berkeley,NaN,NaN
154,570-512,Solano Ave,1799 Solano Avenue Berkeley CA,Berkeley,NaN,NaN
155,17877-164526,Safeway - Berkeley #691,1444 Shattuck Place Berkeley CA,Berkeley,NaN,NaN
156,19864-202264,Telegraph & Ashby,3001 Telegraph Avenue Berkeley CA,Berkeley,NaN,NaN
157,9217-9253,2128 Oxford St.,2128 Oxford Street Berkeley CA,Berkeley,NaN,NaN


대부분의 스타벅스 매장의 데이터가 존재하지만 버클리의 스타벅스 매장의 정보가 누락

### 2) 결측지점 geocoding으로 위치 확인하기

In [76]:
# geocoder 인스턴스 할당
geolocator = Nominatim(user_agent='GIS_user')

# 지오코딩 함수 정의
def my_geocoder(row):
    point = geolocator.geocode(row).point
    return pd.Series({'Latitude' : point.latitude, 'Longitude' : point.longitude})

# 버클리 지역의 위도와 경도를 포인트 좌표로 받아오기
berkeley_locations = rows_with_missing.apply(lambda x: my_geocoder(x['Address']), axis=1)

# 같은 인덱스를 가진 행의 값 중, 겹치는 열의 값을 덮어씌우기
starbucks.update(berkeley_locations)

# null값이 존재하는지 확인
starbucks.isna().sum()

Store Number    0
Store Name      0
Address         0
City            0
Longitude       0
Latitude        0
dtype: int64

### 3) 버클리 지역의 스타벅스 매장 위치 확인하기

In [102]:
# 배경맵 설정하기
m_2 = folium.Map(location=[37.88, -122.26], tiles='openstreetmap', zoom_start=10)

# 버클리 스타벅스 매장 표현하기
for idx, row in starbucks[starbucks['City'] == 'Berkeley'].iterrows():
    Marker([row['Latitude'], row['Longitude']]).add_to(m_2)
    
m_2

### 4) 데이터 통합하기
`CA_counties`데이터를 불러와서 "주(county)"별로 면적을 계산하기

In [81]:
CA_counties = gpd.read_file("../../01_data/CA_county_boundaries/CA_county_boundaries/CA_county_boundaries.shp")
CA_counties.set_crs(epsg=4326)
CA_counties.head()

,GEOID,name,area_sqkm,geometry
0,6091,Sierra County,2491.995494,"POLYGON ((-120.6556 39.69357, -120.65554 39.69..."
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7..."
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822..."
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3..."
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360..."


3가지 데이터프레임을 새롭게 생성
- `CA_pop` : 각 county별로 추정 인구수 데이터프레임
- `CA_high_earners` : 연 소득 $150,000 이상인 사람의 수
- `CA_median_age` : 각 county별로 연령대의 중위수 값

In [84]:
CA_pop = pd.read_csv("../../01_data/CA_county_population.csv", index_col="GEOID")
CA_high_earners = pd.read_csv("../../01_data/CA_county_high_earners.csv", index_col="GEOID")
CA_median_age = pd.read_csv("../../01_data/CA_county_median_age.csv", index_col="GEOID")

CA_median_age.head()

,median_age
GEOID,
6001,37.3
6003,44.9
6005,50.6
6007,36.9
6009,51.6


In [87]:
# 데이터 병합
cols_to_add = CA_pop.join([CA_high_earners, CA_median_age]).reset_index()

# 경계 데이터와 병합
CA_stats = CA_counties.merge(cols_to_add, on='GEOID')

CA_stats.head(2)

,GEOID,name,area_sqkm,geometry,population,high_earners,median_age
0,6091,Sierra County,2491.995494,"POLYGON ((-120.6556 39.69357, -120.65554 39.69...",2987,111,55.0
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7...",1540975,65768,35.9


면적대비 인구수 비율을 확인하기 위해 인구밀집도 컬럼을 생성

In [88]:
CA_stats['density'] = CA_stats['population'] / CA_stats['area_sqkm']

CA_stats.head(2)

,GEOID,name,area_sqkm,geometry,population,high_earners,median_age,density
0,6091,Sierra County,2491.995494,"POLYGON ((-120.6556 39.69357, -120.65554 39.69...",2987,111,55.0,1.198638
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7...",1540975,65768,35.9,598.376878


## 2-2 분석 진행

### 1) 어떤 county(주) 가 유력한지 파악
모든 정보를 하나의 데이터프레임으로 통합하여 특정 기준을 만족하는 주를 선택 \
새로운 데이터프레임은 `sel_counties`라는 데이터프레임으로 저장 \
> **기본 조건 (모든 항목 만족해야 함):**
- 연소득 15만 달러 이상 가구 수가 최소 100,000세대 이상
- 중위 연령(Median Age)이 38.5세 미만
- 인구 밀도가 평방킬로미터당 285명 이상

> **추가 조건 (다음 중 최소 1가지 이상 만족해야 함):**
- 연소득 15만 달러 이상 가구 수가 500,000세대 이상, 또는
- 중위 연령이 35.5세 미만, 또는
- 인구 밀도가 평방킬로미터당 1,400명 이상


In [90]:
sel_counties = CA_stats[(CA_stats['high_earners'] >= 100000) &
                 (CA_stats['median_age'] < 38.5) &
                 (CA_stats['density'] >= 285) &
                 ((CA_stats['high_earners'] >= 500000) |
                  (CA_stats['median_age'] < 35.5) |
                  (CA_stats['density'] >= 1400))]

sel_counties

,GEOID,name,area_sqkm,geometry,population,high_earners,median_age,density
5,6037,Los Angeles County,12305.376879,"MULTIPOLYGON (((-118.66761 33.47749, -118.6682...",10105518,501413,36.0,821.227834
8,6073,San Diego County,11721.342229,"POLYGON ((-117.43744 33.17953, -117.44955 33.1...",3343364,194676,35.4,285.237299
10,6075,San Francisco County,600.588247,"MULTIPOLYGON (((-122.60025 37.80249, -122.6123...",883305,114989,38.3,1470.733077


### 2) 조건을 만족하는 county(주)에 스타벅스 매장 현황 파악하기

In [94]:
starbucks_gdf = gpd.GeoDataFrame(starbucks, geometry=gpd.points_from_xy(starbucks.Longitude, starbucks.Latitude))
starbucks_gdf.set_crs(epsg=4326)
starbucks_gdf.head(2)

,Store Number,Store Name,Address,City,Longitude,Latitude,geometry
0,10429-100710,Palmdale & Hwy 395,14136 US Hwy 395 Adelanto CA,Adelanto,-117.40,34.51,POINT (-117.4 34.51)
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16,POINT (-118.76 34.16)


In [98]:
# 스타벅스 매장의 GeoDataFrame과 조건에 맞는 주 데이터프레임을 공간조인하기
locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties)
print(f"조건에 맞는 주에 존재하는 스타벅스 매장의 수 : {len(locations_of_interest)}개")

조건에 맞는 주에 존재하는 스타벅스 매장의 수 : 1043개


In [103]:
# 시각화 표현하기
mc = MarkerCluster()

for idx, row in locations_of_interest.iterrows():
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc.add_child(folium.Marker([row['Latitude'], row['Longitude']]))
        
m_2.add_child(mc)